## Correlation Prediction
#### Solve correlation prediction with convolutional neural networks(CNN).

In [ ]:
# check GPU type.
!nvidia-smi

### Import Packages

In [2]:
_exp_name = "sample"

In [3]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import efficientnet_v2_s
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
import random

/home/chengxi/anaconda3/envs/p/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

### Transforms

In [5]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    
    # transforms.RandomResizedCrop((128,128), scale=(0.7,1.0)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # #transforms.RandomRotation(20),
    # transforms.RandomAffine(30),
    #transforms.RandomPerspective(distortion_scale=0.2, p=0.2, fill=0),
    # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    # transforms.RandomGrayscale(p=0.2),
    #transforms.GaussianBlur(7, sigma=(0.1, 2.0)),
    
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
    #transforms.Normalize([0.4766, 0.4527, 0.3926], [0.2275, 0.2224, 0.2210]),
])

### Datasets

In [6]:
class CorrDataset1(Dataset):

    def __init__(self,path, csv_path, tfm=test_tfm, files = None):
        super(CorrDataset1).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".png")])
        if files != None:
            self.files = files
            
        self.transform = tfm
               
        df = pd.read_csv(csv_path)
        self.label_map = dict(zip(df['id'], df['corr']))

  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname).convert("RGB")
        im = self.transform(im)
        
        if self.label_map is not None:
            try:
                value = float(self.label_map[os.path.splitext(fname)[0].replace('./correlation_assignment/train/', '')])
            except KeyError:
                value = -1.0
        else:
            value = -1.0
            
        return im,value

In [7]:
class CorrDataset2(Dataset):

    def __init__(self,path, csv_path, tfm=test_tfm, files = None):
        super(CorrDataset2).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".png")])
        if files != None:
            self.files = files
            
        self.transform = tfm
               
        df = pd.read_csv(csv_path)
        self.label_map = dict(zip(df['id'], df['corr']))

  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname).convert("RGB")
        im = self.transform(im)
        
        if self.label_map is not None:
            try:
                value = float(self.label_map[os.path.splitext(fname)[0].replace('./correlation_assignment/valid/', '')])
            except KeyError:
                value = -1.0
        else:
            value = -1.0
            
        return im,value

### Model

In [8]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()

        # Load pretrained EfficientNetB0
        self.base_model = efficientnet_v2_s(weights = None)

        # Get number of features in last layer
        in_features = self.base_model.classifier[1].in_features

        # Replace the classifier with custom regression head
        self.base_model.classifier = nn.Sequential(
            nn.Linear(in_features, 256),  # First dense layer
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),           # Optional second dense layer
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.base_model(x)

### Configurations

In [11]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Regressor().to(device)

# The number of batch size.
batch_size = 128

# The number of training epochs.
n_epochs = 100

# If no improvement in 'patience' epochs, early stop.
patience = 20

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.MSELoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

# K-fold
k_folds = 5

### Dataloader

In [10]:
# Construct train and valid datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = CorrDataset1("./correlation_assignment/train", "./correlation_assignment/responses.csv", tfm=train_tfm)
valid_set = CorrDataset2("./correlation_assignment/valid", "./correlation_assignment/responses.csv", tfm=test_tfm)
dataset = ConcatDataset([train_set, valid_set])

kfold = KFold(n_splits=k_folds, shuffle=True)
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_subsampler, shuffle=False, num_workers=0, pin_memory=True)
    valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_subsampler, shuffle=False, num_workers=0, pin_memory=True)

### Start Training

In [12]:
# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_loss = float('inf')

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, values = batch

        # Forward the data. (Make sure data and model are on the same device.)
        preds = model(imgs.to(device))
        preds = preds.float()
        values = values.float()

        loss = criterion(preds, values.to(device).view(-1, 1))
        
        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Record the loss.
        train_loss.append(loss.item())
        
    train_loss = sum(train_loss) / len(train_loss)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.8f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_losses = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, values = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            preds = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(preds, values.to(device).view(-1, 1))

        # Record the loss.
        valid_losses.append(loss.item())


    # The average loss for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_losses) / len(valid_losses)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.8f}")

    # save models
    if valid_loss < best_loss:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_loss = valid_loss
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

100%|██████████| 250/250 [01:02<00:00,  4.03it/s]


[ Train | 001/100 ] loss = 0.02483268


100%|██████████| 63/63 [00:09<00:00,  6.76it/s]


[ Valid | 001/100 ] loss = 0.00616661
Best model found at epoch 0, saving model


100%|██████████| 250/250 [00:52<00:00,  4.76it/s]


...

[ Valid | 065/100 ] loss = 0.00025118


100%|██████████| 250/250 [01:01<00:00,  4.08it/s]


[ Train | 066/100 ] loss = 0.00086776


100%|██████████| 63/63 [00:08<00:00,  7.61it/s]

[ Valid | 066/100 ] loss = 0.00022492
No improvment 20 consecutive epochs, early stopping


### Dataloader for test

In [ ]:
class CorrDataset(Dataset):

    def __init__(self,path, csv_path, tfm=test_tfm, files = None):
        super(CorrDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".png")])
        if files != None:
            self.files = files
            
        self.transform = tfm
               
        df = pd.read_csv(csv_path)
        self.label_map = dict(zip(df['id'], df['corr']))

  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname).convert("RGB")
        im = self.transform(im)
        
        if self.label_map is not None:
            try:
                value = float(self.label_map[os.path.splitext(fname)[0].replace('./correlation_assignment/images/', '')])
            except KeyError:
                value = -1.0
        else:
            value = -1.0
            
        return im,value,fname

In [ ]:
# Construct test datasets.
# The argument "loader" tells how torchvision reads the data.
test_set = CorrDataset("./correlation_assignment/test", "./correlation_assignment/responses.csv", tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=0, pin_memory=True)

### Dirty Data Cleaning

In [ ]:
import shutil

model_best = Regressor().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()

loss_fn = nn.L1Loss(reduction='mean')

target_dir = './correlation_assignment/clean'

img_clean = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        imgs, values, fname = batch
        print(fname)
        # print(imgs)

        preds = model(imgs.to(device))
        preds = preds.float()
        values = values.float()
        
        loss = loss_fn(preds, values.to(device))
        print(loss.item())
        if (loss.item() <= 0.3):
            filename = os.path.basename(fname[0])  # Extracts 'image1.jpg'
            target_path = os.path.join(target_dir, filename)
            shutil.copy2(fname[0], target_path)  # copy2 preserves metadata




NameError: name 'Regressor' is not defined

## Testing

In [ ]:
model_best = Regressor().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()

test_losses = []

# Iterate the validation set by batches.
for batch in tqdm(valid_loader):

    # A batch consists of image data and corresponding labels.
    imgs, values = batch

    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        preds = model(imgs.to(device))

    loss = criterion(preds, values.to(device))

    test_losses.append(loss.item())

# The average loss for entire validation set is the average of the recorded values.
test_loss = sum(test_losses) / len(test_losses)

# Print the information.
print(f"[ Test ] loss = {test_loss:.8f}")